In [6]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm
from sklearn.utils.class_weight import compute_class_weight

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render
from ax.utils.tutorials.cnn_utils import train, evaluate

from imblearn.over_sampling import SMOTE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
data_train_path = 'skin-lesions/train/'
data_valid_path = 'skin-lesions/valid/'
data_test_path = 'skin-lesions/test/'

https://discuss.pytorch.org/t/is-there-a-limit-on-how-disbalanced-a-train-set-can-be/26334/6?u=ptrblck

In [4]:
# train_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(p = 0.5),
#     transforms.RandomRotation(degrees = 20),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])
# ])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])

train_dataset = torch.load('model_data/train_dataset.pt')
# labels = []
# for idx, (sample, target) in enumerate(tqdm(train_dataset, total=len(train_dataset))):
#     labels.append(int(target))
# cls_weights = torch.from_numpy(
#     compute_class_weight('balanced', classes=np.unique(labels), y=labels)
# )

# weights = cls_weights[labels]
# sampler = WeightedRandomSampler(weights, len(labels), replacement=True)

# train_dataset = ImageFolder(data_train_path, transform = train_transform)

valid_dataset = ImageFolder(data_valid_path, transform = test_transform)
test_dataset = ImageFolder(data_test_path, transform = test_transform)

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle = True)

In [7]:
modified_outputs, labels = [], []
for idx, (sample, target) in enumerate(tqdm(train_dataset, total=len(train_dataset))):
    modified_outputs.append(sample.cpu().detach().numpy())
    labels.append(target)

X_train, y_train = np.array(modified_outputs), np.array(labels)

sm = SMOTE(random_state=42)

train_rows=len(X_train)
X_train = X_train.reshape(train_rows,-1)

X_train, y_train = sm.fit_resample(X_train, y_train)
X_train = X_train.reshape(-1, 3, 224, 224)

train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [16]:
def train(model, epochs, criterion, min_loss, optimizer, vectorize=False):
    training_losses, valid_losses, accs = [],[],[]
    for epoch in range(epochs):
        training_loss = 0
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            if vectorize:
                images = images.reshape(-1, 224 * 224)

            optimizer.zero_grad()
            ps = model(images)
            labels = labels.reshape(len(labels), 1)
            loss = criterion(ps, labels.float())
            loss.backward()
            optimizer.step()

            training_loss += loss.item()
        print(f"\tEPOCH: {epoch + 1}.. TRAINING LOSS: {training_loss}")

        training_losses.append(training_loss)
        model.eval()
        valid_loss = 0
        acc = 0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                
                if vectorize:
                    images = images.reshape(-1, 224 * 224 * 3)
                
                optimizer.zero_grad()
                ps = model(images)
                labels = labels.reshape(len(labels), 1)
                loss = criterion(ps, labels.float())
                
                valid_loss += loss.item()
                
                _, top_class = ps.topk(1, dim = 1)
                eq = top_class == labels.view(-1, 1)
                acc += eq.sum().item()
                
        valid_losses.append(valid_loss)
        accs.append(acc)
        acc = (acc/len(valid_dataset)) * 100
        print("EPOCHS: {}/{}.. \tTRAINING LOSS: {:.6f}.. \tVALIDATION LOSS: {:.6f}.. \tACCURACY: {:.2f}%..".format(epoch + 1, epochs, training_loss, valid_loss, acc))
        
        if valid_loss <= min_loss:
            print("Saving Model {:.4f} ---> {:.4f}".format(min_loss, valid_loss))
            save_obj = OrderedDict([
                ("min_loss", valid_loss),
                ("model", model.state_dict())
            ])
            torch.save(save_obj, "/melanoma_model.pt")
            min_loss = valid_loss
            
    return training_losses, valid_losses, accs

In [14]:
class LogisticRegression(torch.nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(224 * 224 * 3, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = x.reshape(-1, 224 * 224 * 3)
        outputs = self.model(x)
        return outputs

In [37]:
def init_net(parameterization):

    model = LogisticRegression().to(device)

    for param in model.parameters():
        param.requires_grad = False # Freeze feature extractor
        
    return model # return untrained model


def net_train(net, train_loader, parameters, dtype, device):
    net.to(dtype=dtype, device=device)

    # Define loss and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.SGD(net.parameters(), # or any optimizer you prefer 
                        lr=parameters.get("lr", 0.001), # 0.001 is used if no lr is specified
                        momentum=parameters.get("momentum", 0.9)
    )

    scheduler = optim.lr_scheduler.StepLR(
      optimizer,
      step_size=int(parameters.get("step_size", 30)),
      gamma=parameters.get("gamma", 1.0),  # default is no learning rate decay
    )

    num_epochs = parameters.get("num_epochs", 3) # Play around with epoch number
    # Train Network
    for _ in range(num_epochs):
        for inputs, labels in train_loader:
            # move data to proper dtype and device
            inputs = inputs.to(dtype=dtype, device=device)
            inputs = inputs.reshape(-1, 224 * 224 * 3)
            
            labels = labels.to(device=device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.requires_grad=True
            loss.backward()
            optimizer.step()
            scheduler.step()
        return net


def train_evaluate(parameterization):

    # constructing a new training data loader allows us to tune the batch size
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                batch_size=parameterization.get("batchsize", 32),
                                sampler=sampler)
        
    # Get neural net
    untrained_net = init_net(parameterization)
    
    # train
    trained_net = net_train(net=untrained_net, train_loader=train_loader, 
                            parameters=parameterization, dtype=torch.float, device=device)
    
    # return the accuracy of the model as it was trained in this run
    return evaluate(
        net=trained_net,
        data_loader=test_loader,
        dtype=dtype,
        device=device,
    )


In [ ]:
best_parameters, values, experiment, model = optimize(
    parameters=[
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.4], "log_scale": True},
        {"name": "batchsize", "type": "range", "bounds": [16, 128]},
        {"name": "momentum", "type": "range", "bounds": [0.0, 1.0]},
#         {"name": "max_epoch", "type": "range", "bounds": [1, 30]},
#         {"name": "stepsize", "type": "range", "bounds": [20, 40]},        
    ],
  
    evaluation_function=train_evaluate,
    objective_name='accuracy',
)

print(best_parameters)
means, covariances = values
print(means)
print(covariances)

In [17]:
torch.cuda.empty_cache()

model = LogisticRegression().to(device)

LEARNING_RATE = 1e-4
EPOCHS = 50
optimizer = optim.SGD(model.parameters(), lr = LEARNING_RATE)
criterion = nn.BCELoss()

min_loss = -1
train_loss, valid_loss, accs = train(model, EPOCHS, criterion, min_loss, optimizer, vectorize=True)

	EPOCH: 1.. TRAINING LOSS: 35.40567845106125
EPOCHS: 1/50.. 	TRAINING LOSS: 35.405678.. 	VALIDATION LOSS: 3.092602.. 	ACCURACY: 20.00%..
	EPOCH: 2.. TRAINING LOSS: 35.27925366163254
EPOCHS: 2/50.. 	TRAINING LOSS: 35.279254.. 	VALIDATION LOSS: 3.789863.. 	ACCURACY: 20.00%..
	EPOCH: 3.. TRAINING LOSS: 35.3453808426857
EPOCHS: 3/50.. 	TRAINING LOSS: 35.345381.. 	VALIDATION LOSS: 2.685122.. 	ACCURACY: 20.00%..
	EPOCH: 4.. TRAINING LOSS: 34.82768899202347
EPOCHS: 4/50.. 	TRAINING LOSS: 34.827689.. 	VALIDATION LOSS: 3.526606.. 	ACCURACY: 20.00%..
	EPOCH: 5.. TRAINING LOSS: 34.64899832010269
EPOCHS: 5/50.. 	TRAINING LOSS: 34.648998.. 	VALIDATION LOSS: 3.059412.. 	ACCURACY: 20.00%..
	EPOCH: 6.. TRAINING LOSS: 34.57590037584305
EPOCHS: 6/50.. 	TRAINING LOSS: 34.575900.. 	VALIDATION LOSS: 2.644466.. 	ACCURACY: 20.00%..
	EPOCH: 7.. TRAINING LOSS: 34.493152499198914
EPOCHS: 7/50.. 	TRAINING LOSS: 34.493152.. 	VALIDATION LOSS: 4.349686.. 	ACCURACY: 20.00%..
	EPOCH: 8.. TRAINING LOSS: 34.24269276857

KeyboardInterrupt: 